<a href="https://colab.research.google.com/github/HerinePamela/Notebook/blob/main/FARMYIELDS_POLYGONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
from folium.plugins import DualMap
import json
import pandas as pd
from shapely.geometry import shape

In [ ]:
!pip install branca
import branca

In [ ]:
geojson_data = '/content/farmyields.geojson'

In [ ]:
m = folium.Map(location=[0.1072, 34.9286], zoom_start=12)  # Set the initial location and zoom

In [ ]:
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Esri.WorldImagery', name="Satellite").add_to(m)

In [ ]:
with open(geojson_data) as f:
    geojson = json.load(f)

In [ ]:
yield_values = [feature['properties'].get('Trt-Ctr_Per_Change') for feature in geojson['features'] if feature['properties'].get('Trt-Ctr_Per_Change') is not None]
if not yield_values:
    raise ValueError("No valid 'farm_yield' values found in the GeoJSON data.")

min_yield = min(yield_values)
max_yield = max(yield_values)

In [ ]:
min_yield, max_yield

(1.6282225237449155, 423.9130434782609)

In [ ]:
import branca.colormap as cm

In [ ]:
colormap = cm.LinearColormap(
    colors=['#e6f5d0', '#b8e186', '#7fbc41', '#4d9221', '#276419'],
    # colors=['#97d7a1', '#78aa7f', '#197516','#104d0f','#0c380b'],# # Light green to dark green
    vmin=min_yield,
    vmax=max_yield,
    caption=" harvest weight change(%)"  # Legend caption
)

In [ ]:
colormap.add_to(m)

FUNCTION TO STYLE THE POLYGONS

In [ ]:
def style_function(feature):
    yield_value = feature['properties'].get('Trt-Ctr_Per_Change')
    if yield_value is None:
        yield_value = min_yield  # Default value for missing data (you can customize this)
    return {
        'fillColor': colormap(yield_value),
        'color': 'black',
        'fill': True,
        'weight': 2,
        'fillOpacity': 0.7,
       'tooltip':f"harvest weight change: {yield_value} %<br>Soil pH: {feature['properties'].get('Mean pH', 'N/A')}",
    }

Function to create a popup with information from GeoJSON properties

In [ ]:
def popup_function(feature):
    harvest_yield = feature['properties'].get('harvest weight change', 'N/A')
    soil_ph = feature['properties'].get('Mean pH', 'N/A')
    return folium.Popup(f"harvest weight change: {harvest_yield} %<br>"
                        f"Soil pH: {soil_ph}")

Add the GeoJSON layer to the map with popups

In [ ]:
geojson_layer = folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["Trt-Ctr_Per_Change", "Mean pH"],
                                  aliases=["harvest weight change %:", "Mean pH:"]),
    popup=folium.GeoJsonPopup(fields=["Trt-Ctr_Per_Change", "Mean pH"],
                              aliases=["harvest weight change %:", "Mean pH:"]),
    name="Farm Data"
).add_to(m)

In [ ]:
# Add title with green text and light green background
title_html = '''
             <h3 align="center" style="font-size:20px; color:green; background-color:#e6f5d0;">
             <b>Percentage farmyield change and Soil pH Analysis</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Save the map with the title embedded into the HTML
m.save('final.html')

Add Layer control to switch between basemaps


In [ ]:
folium.LayerControl().add_to(m)

Visualize the webmap and assess the results

In [ ]:
m

In [ ]:
m.save('harvest_review.html')